In [ ]:
# Code to build a Music Recommendation System using Python and Spotify API

In [ ]:
import requests
import base64

# Hold Spotify Developer Web API Credentials The Client ID identifies the
# application, while the Client Secret is a confidential key used for authentication.
CLIENT_ID = 'b0ae5675885f4c1fa1db528ad0bf04e7'
CLIENT_SECRET = 'fcc9f6f154cd42879b6358928b07dbc5'

# Base64 encode the client ID and client secret to Encode
# to create a secure representation of the credentials
client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())

# Request the access token
token_url = 'https://accounts.spotify.com/api/token'
headers = {'Authorization': f'Basic {client_credentials_base64.decode()}'}
data = {'grant_type': 'client_credentials'}
response = requests.post(token_url, data=data, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Access token obtained successfully.")
else:
    print("Error obtaining access token.")
    exit()

Access token obtained successfully.


In [ ]:
# Install Spotify Library

!pip install spotipy

In [ ]:
#  Function to get music data from any playlist on Spotify

import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth

def get_trending_playlist_data(playlist_id, access_token):
    # Set up Spotipy with the access token
    sp = spotipy.Spotify(auth=access_token)

    # Get the tracks from the playlist
    playlist_tracks = sp.playlist_tracks(playlist_id, fields='items(track(id, name, artists, album(id, name)))')

    # Extract relevant information and store in a list of dictionaries
    music_data = []
    for track_info in playlist_tracks['items']:
        track = track_info['track']
        track_name = track['name']
        artists = ', '.join([artist['name'] for artist in track['artists']])
        album_name = track['album']['name']
        album_id = track['album']['id']
        track_id = track['id']

        # Get audio features for the track
        audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None

        # Get release date of the album
        try:
            album_info = sp.album(album_id) if album_id != 'Not available' else None
            release_date = album_info['release_date'] if album_info else None
        except:
            release_date = None

        # Get popularity of the track
        try:
            track_info = sp.track(track_id) if track_id != 'Not available' else None
            popularity = track_info['popularity'] if track_info else None
        except:
            popularity = None

        # Add additional track information to the track data
        track_data = {
            'Track Name': track_name,
            'Artists': artists,
            'Album Name': album_name,
            'Album ID': album_id,
            'Track ID': track_id,
            'Popularity': popularity,
            'Release Date': release_date,
            'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
            'Explicit': track_info.get('explicit', None),
            'External URLs': track_info.get('external_urls', {}).get('spotify', None),
            'Danceability': audio_features['danceability'] if audio_features else None,
            'Energy': audio_features['energy'] if audio_features else None,
            'Key': audio_features['key'] if audio_features else None,
            'Loudness': audio_features['loudness'] if audio_features else None,
            'Mode': audio_features['mode'] if audio_features else None,
            'Speechiness': audio_features['speechiness'] if audio_features else None,
            'Acousticness': audio_features['acousticness'] if audio_features else None,
            'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
            'Liveness': audio_features['liveness'] if audio_features else None,
            'Valence': audio_features['valence'] if audio_features else None,
            'Tempo': audio_features['tempo'] if audio_features else None,
            # Add more attributes as needed
        }

        music_data.append(track_data)

    # Create a pandas DataFrame from the list of dictionaries
    df = pd.DataFrame(music_data)

    return df

In [ ]:
# Call the Function with a particular playlist ID

playlist_id = '37i9dQZF1DX76Wlfdnj7AP'  # Trending Playlist

# Call the function to get the music data from
# the playlist and store it in a DataFrame
music_df = get_trending_playlist_data(playlist_id, access_token)

# Display the DataFrame
print(music_df)

                     Track Name                                     Artists  \
0                          APT.                            ROSÉ, Bruno Mars   
1         The Emptiness Machine                                 Linkin Park   
2                   Not Like Us                              Kendrick Lamar   
3                   Rah Tah Tah                          Tyler, The Creator   
4                     Big Dawgs                          Hanumankind, Kalmi   
..                          ...                                         ...   
95                        REACT  Switch Disco, Ella Henderson, Robert Miles   
96       Astronaut In The Ocean                                 Masked Wolf   
97  Another Love - Tiësto Remix                           Tom Odell, Tiësto   
98                   Tarantella                            Gabry Ponte, KEL   
99            MONEY ON THE DASH                         Elley Duhé, Whethan   

                     Album Name                Albu

In [ ]:
# Check for null values

print(music_df.isnull().sum())

Track Name          0
Artists             0
Album Name          0
Album ID            0
Track ID            0
Popularity          0
Release Date        0
Duration (ms)       0
Explicit            0
External URLs       0
Danceability        0
Energy              0
Key                 0
Loudness            0
Mode                0
Speechiness         0
Acousticness        0
Instrumentalness    0
Liveness            0
Valence             0
Tempo               0
dtype: int64


In [ ]:
# Building the music recommendation system

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime # Provides classes for manipulating dates and times
from sklearn.metrics.pairwise import cosine_similarity # This function computes the cosine similarity between two sets of vectors (Basically cos theta for 2 Vectors)

data = music_df

In [ ]:
# Function to calculate weighted popularity scores based on release date to recommend the latest releases

def calculate_weighted_popularity(release_date):
    # Convert the release date to datetime object
    release_date = datetime.strptime(release_date, '%Y-%m-%d')

    # Calculate the time span between release date and today's date
    time_span = datetime.now() - release_date

    # Calculate the weighted popularity score based on time span (e.g., more recent releases have higher weight)
    weight = 1 / (time_span.days + 1)

    return weight

In [ ]:
# Normalize the music features using Min-Max scaling

scaler = MinMaxScaler()
music_features = music_df[['Danceability', 'Energy', 'Key',
                           'Loudness', 'Mode', 'Speechiness', 'Acousticness',
                           'Instrumentalness', 'Liveness', 'Valence', 'Tempo']].values
music_features_scaled = scaler.fit_transform(music_features)

In [ ]:
# We will create a hybrid recommendation system for music recommendations. The
# first approach will be based on recommending music based on music audio
# features, and the second approach will be based on recommending music based on
# weighted popularity. The hybrid approach aims to provide more personalized and
# relevant recommendations by considering both the content similarity of songs
# and their weighted popularity.

In [ ]:
# Generate music recommendations based on the music audio features

# a function to get content-based recommendations based on music features
def content_based_recommendations(input_song_name, num_recommendations=5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get the index of the input song in the music DataFrame
    input_song_index = music_df[music_df['Track Name'] == input_song_name].index[0]

    # Calculate the similarity scores based on music features (cosine similarity : a common measure used in content-based filtering)
    similarity_scores = cosine_similarity([music_features_scaled[input_song_index]], music_features_scaled)

    # Get the indices of the most similar songs
    similar_song_indices = similarity_scores.argsort()[0][::-1][1:num_recommendations + 1]

    # Get the names of the most similar songs based on content-based filtering
    content_based_recommendations = music_df.iloc[similar_song_indices][['Track Name', 'Artists', 'Album Name', 'Release Date', 'Popularity']]

    return content_based_recommendations

In [ ]:
# Generate music recommendations based on weighted popularity and combine
# it with the recommendations of the content-based filtering method using
# the hybrid approach

import pandas as pd

def hybrid_recommendations(input_song_name, num_recommendations=5, alpha=0.5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    content_based_rec = content_based_recommendations(input_song_name, num_recommendations)

    popularity_score = music_df.loc[music_df['Track Name'] == input_song_name, 'Popularity'].values[0]

    weighted_popularity_score = popularity_score * calculate_weighted_popularity(
        music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0]
    )

    new_entry = pd.DataFrame({
        'Track Name': [input_song_name],
        'Artists': [music_df.loc[music_df['Track Name'] == input_song_name, 'Artists'].values[0]],
        'Album Name': [music_df.loc[music_df['Track Name'] == input_song_name, 'Album Name'].values[0]],
        'Release Date': [music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0]],
        'Popularity': [weighted_popularity_score]
    })

    hybrid_recommendations = pd.concat([content_based_rec, new_entry], ignore_index=True)

    hybrid_recommendations = hybrid_recommendations.sort_values(by='Popularity', ascending=False)

    hybrid_recommendations = hybrid_recommendations[hybrid_recommendations['Track Name'] != input_song_name]

    return hybrid_recommendations

In [ ]:
# Testing the hybrid recommendation

input_song_name = "I'm Good (Blue)"
recommendations = hybrid_recommendations(input_song_name, num_recommendations=5)
print(f"Hybrid recommended songs for '{input_song_name}':")
print(recommendations)

Hybrid recommended songs for 'I'm Good (Blue)':
                     Track Name                                     Artists  \
1                       Disease                                   Lady Gaga   
2                       KEEP UP                                     Odetari   
3  It's Not Right But It's Okay                            Mr. Belt & Wezol   
4         Rainfall (Praise You)                                   Tom Santa   
0                         REACT  Switch Disco, Ella Henderson, Robert Miles   

                     Album Name Release Date  Popularity  
1                       Disease   2024-10-25        86.0  
2          KEEP UP // FROSTBITE   2024-07-17        84.0  
3  It's Not Right But It's Okay   2024-02-23        78.0  
4         Rainfall (Praise You)   2022-02-18        71.0  
0                         REACT   2023-01-13        70.0  
